# NLP Project
## Topic extraction
The purpose of this project is to extract topics from news articles.

### Step-by-step Process
1. Find a suitable NLP model to use for topic extraction: LDA
   - Cluster texts to discover latent topics
   - Process words and assign them weights indicating their contribution to a topic
2. Preprocess the data
3. Get results
4. Documentation

In [13]:
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short, stem_text
import gensim  # necessary?
from gensim import corpora
from gensim.models import Word2Vec, TfidfModel, LdaMulticore, CoherenceModel
import spacy
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pyLDAvis
import pyLDAvis.gensim_models

### Data Pre-processing

In [3]:
# Large dataset model:
# from sklearn.model_selection import train_test_split

# # read in data
# df_1 = pd.read_csv('Data/articles1.csv')['content'].to_frame()  # only get content-column
# df_2 = pd.read_csv('Data/articles2.csv')['content'].to_frame()
# df_3 = pd.read_csv('Data/articles3.csv')['content'].to_frame()
# df = df_1.append(df_2).append(df_3)
# print('\nData set, shape:', df.shape)
# print(df.head(5))

# # check for missing data
# print(df.isna().sum())  # shows no null values in content-column

# # split data into ~67% training and ~33% testing
# train, test = train_test_split(df, test_size=0.33, random_state=1)
# print('\nTraining data set, shape:', train.shape)
# print('Testing data set, shape:', test.shape)

# # reset indices
# train = train.reset_index(drop=True)
# test = test.reset_index(drop=True)
# print(train.head(5))
# print('\n', test.head(5))

# Small dataset model:
# read in data
df = pd.read_csv('Data/articles1.csv')['content'].to_frame()
# df.drop(df.index[0:49900],0,inplace=True)  # drop a few rows to make dataset smaller and more manageable
print('\nData set, shape:', df.shape)
# check for missing data
print(df.isna().sum())  # shows no null values in content-column

pd.set_option('display.max_colwidth', None)
print(df.head(1))


Data set, shape: (50000, 1)
content    0
dtype: int64
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### Pre-process the Data
Normalisation, cleaning, etc.

In [4]:
# load spacy nlp pre-processing pipeline to use for lemmatization
nlp = spacy.load('en_core_web_sm')

In [5]:
# create filter for gensim nlp pre-processing pipeline to include all steps except stemmatization
CUSTOM_FILTERS = [lambda x: x.lower(),  # lowercase
                  strip_tags,
                  strip_punctuation,  # replace punctuation with whitespace
                  strip_multiple_whitespaces,  # remove repeating whitespaces
                  strip_numeric,  # remove numbers
                  remove_stopwords,  # remove stopwords
                  strip_short,  # remove words with less than 3 characters
                  #  stem_text  # return porter-stemmed text,
                 ]

In [6]:
sample = "Hello, my name is something you'll never guess, Kim! ...But I wrote my signature. Right! My parents called me this, what can I say?"
print(sample)

Hello, my name is something you'll never guess, Kim! ...But I wrote my signature. Right! My parents called me this, what can I say?


In [7]:
# test sample string without filtered pipeline, i.e., with stemmatizer
test_a = preprocess_string(sample)
print(test_a)

['hello', 'guess', 'kim', 'wrote', 'signatur', 'right', 'parent', 'call']


In [8]:
# test sample string with filtered pipeline and lemmatizer
test_b = ' '.join(preprocess_string(sample, CUSTOM_FILTERS))  # pre-process without stemmatizing
lem = [token.lemma_ for token in nlp(test_b)]  # lemmatize
print(lem)

['hello', 'guess', 'kim', 'write', 'signature', 'right', 'parent', 'call']


In [9]:
def preprocess_articles(x):
    prep = ' '.join(preprocess_string(x, CUSTOM_FILTERS))
    return [token.lemma_ for token in nlp(prep)]

In [10]:
# apply final pipeline to all data
df['preprocessed'] = df['content'].apply(preprocess_articles)
df

content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

### Prepare pre-processed articles for topic extraction models

In [11]:
# create corpus: article contents embedded according to BoW-model using dictionary
article_dict = corpora.Dictionary(df['preprocessed'])
article_dict.filter_extremes(no_below=3)
bow_corpus = [article_dict.doc2bow(line) for line in df['preprocessed']]
tfidf = TfidfModel(bow_corpus)  # take term-frequency into account in bow-model, fit
tfidf_corpus = tfidf[bow_corpus]  # transform

### LDA - Topic Modelling

In [14]:
np.random.seed(123456)
lda_model = LdaMulticore(corpus=tfidf_corpus,
                         id2word=article_dict,
                         random_state=100,
                         num_topics=20)

# tune the number of topics!

In [15]:
# save the model
lda_model.save("LDA_model/LDA_model")

# load the model
# lda_model = LdaModel.load("LDA_model/LDA_model")

In [29]:
# keywords per topic
lda_model.print_topics(-1)

doc_lda = lda_model[tfidf_corpus]

# print(lda_model.print_topic(0))
# print('\n', lda_model.print_topic(1))
# print('\n', lda_model.print_topic(2))

[(0,
  '0.005*"trump" + 0.004*"clinton" + 0.003*"\'" + 0.002*"campaign" + 0.002*"republican" + 0.002*"vote" + 0.002*"state" + 0.002*"election" + 0.002*"hillary" + 0.002*"voter"'),
 (1,
  '0.002*"trump" + 0.002*"\'" + 0.002*"clinton" + 0.001*"company" + 0.001*"woman" + 0.001*"breitbart" + 0.001*"percent" + 0.001*"report" + 0.001*"state" + 0.001*"twitter"'),
 (2,
  '0.004*"trump" + 0.003*"\'" + 0.002*"obama" + 0.002*"clinton" + 0.001*"police" + 0.001*"president" + 0.001*"russia" + 0.001*"campaign" + 0.001*"russian" + 0.001*"attack"'),
 (3,
  '0.003*"\'" + 0.003*"trump" + 0.002*"tesla" + 0.002*"police" + 0.001*"clinton" + 0.001*"car" + 0.001*"company" + 0.001*"uber" + 0.001*"state" + 0.001*"attack"'),
 (4,
  '0.003*"trump" + 0.002*"\'" + 0.002*"clinton" + 0.001*"cruz" + 0.001*"president" + 0.001*"obama" + 0.001*"republican" + 0.001*"news" + 0.001*"campaign" + 0.001*"percent"'),
 (5,
  '0.004*"clinton" + 0.004*"trump" + 0.002*"\'" + 0.002*"comey" + 0.002*"poll" + 0.002*"hillary" + 0.001*"e

In [28]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=lda_model, texts=df['preprocessed'], dictionary=article_dict, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print('Coherence Score: ', coherence_score)

# change num of topics to increase coherence score

Coherence Score:  0.3538168453269389


In [47]:
# print topic(s) per article
# for topic in doc_lda:
#     print(topic)

# doc_lda[4]

# print dominant topic(s) of the 20 first articles
for i in range(21):
    print(doc_lda[i])

[(0, 0.31944287), (2, 0.6100168)]
[(3, 0.92681706)]
[(2, 0.37508905), (19, 0.5500415)]
[(2, 0.47618946), (3, 0.32805297), (6, 0.1570544)]
[(15, 0.88514334)]
[(10, 0.8485794)]
[(15, 0.88604254)]
[(6, 0.9156888)]
[(3, 0.36362094), (14, 0.5422399)]
[(13, 0.8945416)]
[(0, 0.7485062), (6, 0.18657108)]
[(2, 0.91100246)]
[(2, 0.26237208), (6, 0.456358), (15, 0.22928055)]
[(12, 0.8926864)]
[(10, 0.6991633), (15, 0.22078957)]
[(2, 0.9264957)]
[(2, 0.2903221), (3, 0.6364859)]
[(0, 0.91516083)]
[(2, 0.91607004)]
[(2, 0.68013567), (5, 0.21629436)]
[(3, 0.40559873), (10, 0.513817)]


In [55]:
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=tfidf_corpus, dictionary=article_dict)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)